In [10]:
import traceback
import sys
import datetime as dtime
import time
import os

from IPython.core import magic_arguments
from IPython.core.magic import line_magic, cell_magic, line_cell_magic, Magics, magics_class

# Context manager that copies stdout and any exceptions to a log file
class DoublePrint(object):
    """Used to save output to sdtout and file.
    Usage:
    
    with DoublePrint('/home/user/log.txt'):
        print("Print stdout + Write to file")
    print("Print to stdout")
    
    """ 
    def __init__(self, filepath=None):  
        if filepath == None:
            cwd = os.getcwd()
            d = dtime.datetime.today()
            filepath = d.strftime(cwd +"/%Y-%M-%d_%H-%M-%S-%m_log.txt")          
        print("Saving logs to file:", filepath)
        self.stdout = sys.stdout
        self.filepath = filepath
        self.file = open(self.filepath, 'w')
    
    def __enter__(self):
        sys.stdout = self

    def __exit__(self, exc_type, exc_value, tb):
        sys.stdout = self.stdout
        if exc_type is not None:
            self.file.write(traceback.format_exc())
        self.file.close()

    def write(self, data):
        self.stdout.write(data)
        if data.strip() != "":
            self.file.write(self.stamp() + data[:1000] + "\n")
            self.file.flush()

    def flush(self):
        pass
    
    def stamp(self):
        d = dtime.datetime.today()
        string = d.strftime("[%Y-%M-%d %H:%M:%S] ")
        return string

@magics_class
class MagicDoublePrint(Magics):

    @cell_magic
    def doubleprint(self, line, cell):
        with DoublePrint():
            self.shell.run_cell(cell)  

ipy = get_ipython()
ipy.register_magics(MagicDoublePrint)

In [12]:
%%doubleprint

print("dupa1")
print("ala2")

Saving logs to file: /home/pawel/projects/DoublePrint/2019-26-29_19-26-43-08_log.txt
dupa1
ala2
